In [1]:
import numpy as np
import pandas as pd
from mysql.connector import connect, Error
from KeywordExtraction import *

import torch
print(torch.cuda.is_available())

True


Load Model

In [2]:
model = torch.load('./model-3.pt')

In [3]:
for i in idx_to_type: print(i,idx_to_type[i])

0 いくら
1 重さ
2 長さ
3 いつ
4 在庫


In [4]:
idx_to_type_query = {0:'price',1:'weight',2:'size',3:'delivery',4:'in_stock'}

In [5]:
question = "ラインディフューザーの料金はいくらですか"
_, item, qtype_id, qtype = predict_keywords(question, model, tokenizer)
print(f"Item in Question: {item}, Question Type: {qtype} = {idx_to_type_query[qtype_id]}")

Item in Question: ラインディフューザー, Question Type: いくら = price


c:\Users\User\miniconda3\envs\torch\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [9]:
try:
    with connect(
        host="localhost",
        user="root",
        password="********",
        database="astem_products"
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f'SELECT product, {idx_to_type_query[qtype_id]} FROM products WHERE product = "{item}";')
            result = cursor.fetchall()
            print('query result: ',result)
            print(f'{item}の {idx_to_type_query[qtype_id]} は {result[0][1]}')
except Error as e:
    print(e)

query result:  [('ラインディフューザー', 3400)]
ラインディフューザーの price は 3400
